In [27]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline

In [28]:
train=pd.read_csv('Train_UWu5bXk.csv')
#,sep=',',na_values=['#NAME?'])
test=pd.read_csv('Test_u94Q5KV.csv')
sample_submit=pd.read_csv('SampleSubmission_TmnO39y.csv')

In [29]:
sample_submit.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,NaN
1,FDW14,OUT017,NaN
2,NCN55,OUT010,NaN
3,FDQ58,OUT017,NaN
4,FDY38,OUT027,NaN


In [30]:
#import pandas profiling
train.columns
test.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [31]:
train.columns.difference(test.columns)

Index(['Item_Outlet_Sales'], dtype='object')

In [32]:
print(train.shape)
print(test.shape)

(8523, 12)
(5681, 11)


In [9]:
!pip install pandas_profiling
#import pandas_profiling

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
train.head

In [12]:
import pandas_profiling as pp

In [33]:
train['source']='train'
test['source']='test'

In [34]:
dataf=pd.concat([train,test],axis=0,sort=False)

In [35]:
dataf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14204 entries, 0 to 5680
Data columns (total 13 columns):
Item_Identifier              14204 non-null object
Item_Weight                  11765 non-null float64
Item_Fat_Content             14204 non-null object
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  10188 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Type                  14204 non-null object
Item_Outlet_Sales            8523 non-null float64
source                       14204 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 1.5+ MB


In [36]:
dataf.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [37]:
dataf.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [21]:
dataf.nunique()

Item_Identifier               1559
Item_Weight                    415
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      3
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3493
source                           2
dtype: int64

In [22]:
pp.ProfileReport(dataf)

Number of variables,14
Number of observations,14204
Total Missing (%),6.1%
Total size in memory,1.5 MiB
Average record size in memory,112.0 B
Numeric,6
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [38]:
dataf['Item_Weight']=dataf.groupby('Item_Identifier').Item_Weight.transform(lambda x: x.fillna(x.mean()))
#dataf['Item_Weight'].unique()

In [39]:
dataf.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [ ]:
def applymode(x):
    a=x.mode()[0]
    print(a)
    return x.fillna(a)

In [25]:
dataf['Outlet_Size']=dataf.groupby('Outlet_Identifier').Outlet_Size.transform(lambda x: x.fillna(x.mode))

In [26]:
dataf.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [41]:
dataf.loc[dataf.Item_Visibility==0.000000,'Item_Visibility']=np.nan

In [ ]:
from scipy.stats import mode
dataf['Item_Visibility']=dataf.groupby('Item_Identifier').Item_Visibility.transform(lambda x: x.fillna(x.mode))

In [42]:
dataf.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility               879
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [43]:
dataf.Item_Fat_Content.value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [46]:
dataf['Item_Fat_Content']=dataf.Item_Fat_Content.replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [47]:
dataf.Item_Fat_Content.value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [48]:
dataf['Total_Years_Outlet']=2018-dataf['Outlet_Establishment_Year']
dataf.drop(columns='Outlet_Establishment_Year',inplace=True)
dataf.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Total_Years_Outlet
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380,train,19
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228,train,9
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700,train,19
3,FDX07,19.20,Regular,NaN,Fruits and Vegetables,182.0950,OUT010,NaN,Tier 3,Grocery Store,732.3800,train,20
4,NCD19,8.93,Low Fat,NaN,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052,train,31


In [52]:
dataf['Item_Identifier_Pattern']=dataf['Item_Identifier'].apply(lambda x: x[0:2])
dataf['Item_Identifier'].apply(lambda x: x[0:2]).value_counts()

FD    10201
NC     2686
DR     1317
Name: Item_Identifier, dtype: int64

In [53]:
dataf.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Total_Years_Outlet,Item_Identifier_Pattern
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380,train,19,FD
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228,train,9,DR
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700,train,19,FD
3,FDX07,19.20,Regular,NaN,Fruits and Vegetables,182.0950,OUT010,NaN,Tier 3,Grocery Store,732.3800,train,20,FD
4,NCD19,8.93,Low Fat,NaN,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052,train,31,NC


In [ ]:
dataf.Item_Type

In [57]:
dataf.loc[dataf.Outlet_Type.str.startswith('Supermarket'),'Outlet_Type_Pattern']='SuperMarket'

In [58]:
dataf.loc[dataf.Outlet_Type.str.startswith('Grocery'),'Outlet_Type_Pattern']='Grocery'

In [59]:
dataf.Outlet_Type_Pattern.value_counts()

SuperMarket    12399
Grocery         1805
Name: Outlet_Type_Pattern, dtype: int64

In [60]:
data_column_category=dataf.select_dtypes(exclude=[np.number]).columns

In [61]:
data_column_integer=dataf.columns.difference(data_column_category)

In [ ]:
dataf_one_hot=pd.get_dummies(dataf[data_column_category].drop(columns=)

In [ ]:
train_modified=data_model.loc[data_model.source_train==1.0,:]

In [ ]:
test_modified=data_model.loc[data_model.source_train==0.0,:]

In [ ]:
train_modified.drop(columns=['source'])

In [ ]:
train_modified.to_csv=

In [ ]:
train=pd.read_csv('test3')